# **<center>Preprocessing**

### Import Packages

In [47]:
import os
from dotenv import load_dotenv

import pandas as pd
import numpy as np

import datetime as dt
from tqdm import tqdm

### Set Directory and Preferences

In [48]:
# Load in the dotenv variables
load_dotenv()

# Grab the project path from the dotenv file without the addons
project_path = os.getenv('Project_Path')[2:78]

# Change notebook directory back one so that it can acess the data
os.chdir(project_path)

# Show all the columns in the .head() method
pd.set_option('display.max_columns', None)

### Load the Data

In [49]:
# Load the cleaned data from data wrangling
data = pd.read_csv('./data/interim/wrangled', low_memory = False)
# Load in the loan_status column for use
loan = pd.read_csv('./data/raw/loan.csv',low_memory = False)
# Load the original dataset to get the accurate representation
y = loan['loan_status']

### Split the data

Only a part of the dataset has loans that have finished. I can separate out the finished loans from the unfinished loans by the `loan_status` column that describes the current state of the loan.

In [50]:
data['completed'] = np.where(y.isin(['Fully Paid', 'Charged Off',
                                     'Does not meet the credit policy. Status:Fully Paid',
                                     'Does not meet the credit policy. Status:Charged Off']), 1, 0)
loan['completed'] = data['completed']

While the dataset that I'm going to used has been fully cleaned, I need to use the unscaled numbers from the original dataset to calculate certain values.

In [51]:
# replace the string values with numerical values, use a dictionary to be concise
loan['term'].replace({' 36 months':3, ' 60 months':5}, inplace = True)

Now I can split both the wrangled and original data into finished and active datasets. I use the `.copy()` to avoid the copy setting warning.

In [66]:
# Create a dataframe of finished loans that are a copy
finished = data.loc[data['completed'] == 1, :'addr_state_VA'].copy()
# Create a dataframe of the still active loans are a copy
active = data.loc[data['completed'] == 0, :'addr_state_VA'].copy()

# Create a dataframe of the finished loans for the original data as a copy
finished_loan = loan.loc[loan['completed'] == 1, :].copy()
# Create a dataframe of active loans for the original data as a copy
active_loan = loan.loc[loan['completed'] == 0, :].copy()

## Calculating 'frac'

Originally I had thought that `loan_status` was the target variable but the active loans had too much information that came after the time of origination. I pivotted to trying find the amount that the loans are worth for active loans. To do this I can calculate the expected return on each loan from lending club. Lending club does not compound interest for its loans so the formula is:
$$ \text{Expected Payment} = Principal * (1 + \text{Interest Rate}) * years $$

I can then calculate the amount that has been paid back by this formula:
$$ \text{Actual Payment} = \text{Total Payment} + \text{Total Late Fees} + \text{Recoveries} $$
Where `Total Payment` is the amount that has been payed back from the loan, `Total Late fees` are the late fees that the borrower would have incurred if they were late with a fee. `Recoverie`s is everything that that the company was able to recover after the loan was charged off.

`frac` is the fraction in which 
$$ \text{frac} = \frac{\text{Actual Payment}}{\text{Expected Payment}}


In [67]:
finished_loan['int_rate_usable'] = finished_loan['int_rate'] / 100 + 1

finished['$_expected'] = finished_loan['funded_amnt'] * finished_loan['int_rate'] * finished_loan['term']

finished['$_actual'] = finished_loan[['total_pymnt', 'total_rec_late_fee', 'total_rec'
                                      'recoveries']].sum(axis = 1) 

finished['frac'] = finished['$_actual'] / finished['$_expected']

finished.drop(['$_expected','$_actual'], axis = 1, inplace = True)

The active loans are slightly different in that the expected payment is what is remaining on the loan instead of the entire loan. To calculate this I did:
$$ \text{Expected Payment} = Principal * (1 + \text{Interest Rate}) * years - \text{Actual Payment} $$
Where `Actual Payment' is the same as the cell above. The idea is that the expected payment is the entire loan's expected value minus the money that the borrower has already paid. 


In [77]:
active_loan['int_rate_usable'] = active_loan['int_rate'] / 100 + 1

active['$_expected'] = active_loan['funded_amnt'] * active_loan['int_rate'] * active_loan['term'] - active_loan['total_pymnt']

active.drop('int_rate_usable', axis = 1, inplace = True)

With the `frac` column added, I can now save the datasets for modelling.

In [81]:
finished.to_csv('./data/processed/train.csv', index = False)

active.to_csv('./data/processed/predict.csv', index = False)